### Passo 1 - Importar Arquivos e Bibliotecas

In [1]:
#importar bibliotecas
import pandas as pd
import win32com.client as win32
import pathlib

In [2]:
#importar base de dados:
emails = pd.read_excel(r'Bases de Dados\Emails.xlsx')
lojas = pd.read_csv(r'Bases de Dados\Lojas.csv', encoding='latin1', sep=';')
vendas = pd.read_excel(r'Bases de Dados\Vendas.xlsx')

### Passo 2 - Definir Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [3]:
#incluir nome da loja em planilha vendas:
vendas = vendas.merge(lojas, on='ID Loja')

#criar tabela para cada loja
dic_lojas = {}
for loja in lojas['Loja']:
    dic_lojas[loja] = vendas.loc[vendas['Loja']==loja , :]

#dia mais recente para calcular o indicador
dia = vendas['Data'].max()

### Passo 3 - Salvar a planilha na pasta de backup

In [4]:
#identificar pasta existente
caminho = pathlib.Path(r'Backup Arquivos Lojas')
arquivos = caminho.iterdir()
lista_backup = [arquivo.name for arquivo in arquivos]

#criando a pasta
for loja in dic_lojas:
    if loja not in lista_backup:
        nova_pasta = caminho / loja
        nova_pasta.mkdir()
        
#salvar na pasta
    nome = '{}_{}_{}.xlsx'.format(dia.month, dia.day, loja)
    local = caminho / loja / nome
    dic_lojas[loja].to_excel(local)

### Passo 4 - Calcular o indicador para 1 loja

In [5]:
#definição de metas
meta_faturamento_ano = 1650000
meta_faturamento_dia = 1000
meta_qtd_ano = 120
meta_qtd_dia = 4
meta_ticket = 500

### Passo 5 - Enviar por e-mail para o gerente

In [6]:
for loja in dic_lojas:
    vendas_loja = dic_lojas[loja]
    vendas_dia = vendas_loja.loc[vendas_loja['Data']==dia, :]
    
    #indicador faturamento
    faturamento_dia = vendas_dia['Valor Final'].sum()
    faturamento_ano = vendas_loja['Valor Final'].sum()
    
    #indicador diversidade de produtos
    qtd_ano = len(vendas_loja['Produto'].unique())
    qtd_dia = len(vendas_dia['Produto'].unique())
    
    #indicador ticket médio
    valor_venda = vendas_loja.groupby('Código Venda').sum(numeric_only=True)
    valor_dia = vendas_dia.groupby('Código Venda').sum(numeric_only=True)
    
    ticket_ano = valor_venda['Valor Final'].mean()
    ticket_dia = valor_dia['Valor Final'].mean()

    outlook = win32.Dispatch('outlook.application')
    
    nome = emails.loc[emails['Loja']==loja, 'Gerente'].values[0]
    
    mail = outlook.CreateItem(0)
    mail.To = emails.loc[emails['Loja']==loja, 'E-mail'].values[0]
    mail.Subject = f'OnaPage Dia {dia.day}/{dia.month} - Loja {loja}'
    
    #faturamento
    if faturamento_dia >= meta_faturamento_dia:
        cor_fat_dia = 'green' 
    else:
        cor_fat_dia = 'red'
    if faturamento_ano >= meta_faturamento_ano:
        cor_fat_ano = 'green' 
    else:
        cor_fat_ano = 'red'
        
    #produto
    if qtd_dia >= meta_qtd_dia:
        cor_qtd_dia = 'green'
    else:
        cor_qtd_dia = 'red'
    if qtd_ano >= meta_qtd_ano:
        cor_qtd_ano = 'green'
    else:
        cor_qtd_ano = 'red'
        
    #ticket
    if ticket_dia >= meta_ticket:
        cor_ticket_dia = 'green'
    else:
        cor_ticket_dia = 'red'
    if ticket_ano >= meta_ticket:
        cor_ticket_ano = 'green'
    else:
        cor_ticket_ano = 'red'
    
    
    mail.HTMLBody = f'''
    <p>Bom dia, {nome}<\p>
    <p>O resultado de ontem <strong>{dia.day}/{dia.month}<\strong> da Loja <strong>{loja}<\strong><\p>
    
    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cenário Dia</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_dia:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_dia:.2f}</td>
        <td style="text-align: center"><font color="{cor_fat_dia}">◙<\font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtd_dia}</td>
        <td style="text-align: center">{meta_qtd_dia}</td>
        <td style="text-align: center"><font color="{cor_qtd_dia}">◙<\font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">{ticket_dia}</td>
        <td style="text-align: center">{meta_ticket}</td>
        <td style="text-align: center"><font color="{cor_ticket_dia}">◙<\font></td>
      </tr>
    </table>
    <br>
    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Ano</th>
        <th>Meta Ano</th>
        <th>Cenário Ano</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_ano:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_ano:.2f}</td>
        <td style="text-align: center"><font color="{cor_fat_ano}">◙<\font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtd_ano}</td>
        <td style="text-align: center">{meta_qtd_ano}</td>
        <td style="text-align: center"><font color="{cor_qtd_ano}">◙<\font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">{ticket_ano}</td>
        <td style="text-align: center">{meta_ticket}</td>
        <td style="text-align: center"><font color="{cor_ticket_ano}">◙<\font></td>
      </tr>
    </table>
    
    <p>Segue em anexo a planilha com todos os dados para mais detalhes.<\p>
    <p>Qualquer duvida estou à disposição.<\p>
    <p>Att, Guilherme<\p>
    '''
    
    attachment = pathlib.Path.cwd() / caminho / loja / f'{dia.month}_{dia.day}_{loja}.xlsx'
    mail.Attachments.Add(str(attachment))
    
    mail.Send()
    print("Email enviado")

Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado
Email enviado


### Passo 6 - Automatizar todas as lojas

### Passo 7 - Criar ranking para diretoria

In [7]:
faturamento_lojas = vendas.groupby('Loja')[['Loja', 'Valor Final']].sum(numeric_only=True)
faturamento_lojas = faturamento_lojas.sort_values(by='Valor Final', ascending=False)
display(faturamento_lojas)

nome_arquivo = '{}_{}_{}_Ranking_Anual.xlsx'.format(dia.month, dia.day, loja)
faturamento_lojas.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))

vendas_dia = vendas.loc[vendas['Data']==dia, :]
faturamento_lojas_dia = vendas_dia.groupby('Loja')[['Loja', 'Valor Final']].sum(numeric_only=True)
faturamento_lojas_dia = faturamento_lojas_dia.sort_values(by='Valor Final', ascending=False)
display(faturamento_lojas_dia)

nome_arquivo = '{}_{}_{}_Ranking_Dia.xlsx'.format(dia.month, dia.day, loja)
faturamento_lojas.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))

,Valor Final
Loja,
Iguatemi Campinas,1762419
Shopping Vila Velha,1731167
Bourbon Shopping SP,1726110
Rio Mar Recife,1722766
Shopping SP Market,1721763
Palladium Shopping Curitiba,1721120
Norte Shopping,1711968
Ribeirão Shopping,1707122
Iguatemi Esplanada,1699681


,Valor Final
Loja,
Salvador Shopping,3950
Novo Shopping Ribeirão Preto,3400
Center Shopping Uberlândia,2651
Shopping Eldorado,2391
Shopping Center Interlagos,1582
Shopping Recife,1366
Norte Shopping,1259
Shopping União de Osasco,1207
Shopping Vila Velha,937


### Passo 8 - Enviar e-mail para diretoria

In [9]:
#enviar o e-mail
outlook = win32.Dispatch('outlook.application')

mail = outlook.CreateItem(0)
mail.To = emails.loc[emails['Loja']=='Diretoria', 'E-mail'].values[0]
mail.Subject = f'Ranking Dia {dia.day}/{dia.month}'
mail.Body = f'''
Prezados, bom dia

Melhor loja do Dia em Faturamento: Loja {faturamento_lojas_dia.index[0]} com Faturamento R${faturamento_lojas_dia.iloc[0, 0]:.2f}
Pior loja do Dia em Faturamento: Loja {faturamento_lojas_dia.index[-1]} com Faturamento R${faturamento_lojas_dia.iloc[-1, 0]:.2f}

Melhor loja do Ano em Faturamento: Loja {faturamento_lojas.index[0]} com Faturamento R${faturamento_lojas.iloc[0, 0]:.2f}
Pior loja do Ano em Faturamento: Loja {faturamento_lojas.index[-1]} com Faturamento R${faturamento_lojas.iloc[-1, 0]:.2f}

Segue em anexo os rankings do ano e do dia de todas as lojas.

Qualquer dúvida estou à disposição.

Att.,
Guilherme
'''

# Anexos:
# attachment  = pathlib.Path.cwd() / caminho / f'{dia.month}_{dia.day}_Ranking Anual.xlsx'
# mail.Attachments.Add(str(attachment))

# attachment  = pathlib.Path.cwd() / caminho / f'{dia.month}_{dia.day}_Ranking Dia.xlsx'
# mail.Attachments.Add(str(attachment))


mail.Send()
print('E-mail da Diretoria enviado')

E-mail da Diretoria enviado
